In [1]:
import numpy as np


In [2]:
word_vec_dict = {}

with open('glove.6B.50d.txt','r', encoding='utf-8') as f:
    vocab = set()
    
    for line in f:
        line = line.strip().split()
        vocab.add(line[0])
        word_vec_dict[line[0]] = np.array(line[1:], dtype=np.float64)
    

In [3]:
def cosine_similarity(u, v):
    
    norm_u = np.linalg.norm(u)
    norm_v = np.linalg.norm(v)
    
    cos_theta = np.dot(u, v) / (norm_u * norm_v)
    
    return cos_theta



In [4]:
gender_vec = word_vec_dict['woman'] - word_vec_dict['man']

In [7]:
def gender_debias(word, word_vec_dict, gender_vec):
    
    word_embed = word_vec_dict[word]
    embed_bias_component = (np.dot(word_embed, gender_vec) / (np.linalg.norm(gender_vec) ** 2)) * gender_vec
    word_embed_debiased = word_embed - embed_bias_component
    
    return word_embed_debiased
    




    
    
    

In [19]:
def equalize_gender_embeddings(word_pair, gender_vec, word_vec_dict):
    
    word_a, word_b = word_pair
    word_a_embed = word_vec_dict[word_a]
    word_b_embed = word_vec_dict[word_b]
    
    embed_mean = (word_a_embed + word_b_embed) / 2
    
    embed_bias_axis = (np.dot(embed_mean, gender_vec) / np.linalg.norm(gender_vec ** 2)) * gender_vec
    embed_orthogonal_axis = embed_mean - embed_bias_axis
    
    word_a_embed_bias = (np.dot(word_a_embed, gender_vec) / np.linalg.norm(gender_vec ** 2)) * gender_vec
    word_b_embed_bias = (np.dot(word_b_embed, gender_vec) / np.linalg.norm(gender_vec ** 2)) * gender_vec
    
    
    word_a_embed_equalized = (np.sqrt(abs(1-np.linalg.norm(embed_orthogonal_axis) ** 2)) / np.linalg.norm(word_a_embed-embed_orthogonal_axis-embed_bias_axis)) * (word_a_embed_bias - embed_bias_axis)
    word_b_embed_equalized = (np.sqrt(abs(1-np.linalg.norm(embed_orthogonal_axis) ** 2)) / np.linalg.norm(word_b_embed-embed_orthogonal_axis-embed_bias_axis)) * (word_b_embed_bias - embed_bias_axis)
    
    word_a_embed_equalized += embed_orthogonal_axis
    word_b_embed_equalized += embed_orthogonal_axis
    
   
    return word_a_embed_equalized, word_b_embed_equalized